In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from itertools import chain 
from collections import Counter
from bs4 import BeautifulSoup
import string
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy
nlp = spacy.load('en_core_web_lg')
import re
from nltk.tag.perceptron import PerceptronTagger
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, plot_confusion_matrix
from sklearn.feature_extraction.text import CountVectorizer
import requests
import urllib.request
import time
from bs4 import BeautifulSoup

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(filename)
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv("../input/train.tsv", sep="\t")
test = pd.read_csv("../input/test.tsv", sep="\t")
sample = pd.read_csv("../input/sampleSubmission.csv", sep="\t")

In [ ]:
print(train.shape)
print(train.head())

In [ ]:
plt.figure(figsize=(9,6))
sns.countplot(y=train.Sentiment, order = train.Sentiment.value_counts().iloc[:5].index)
plt.title('The distribution of the Sentiment')
plt.show()

In [ ]:
def ent_replace(string):
    '''Function that takes a string as an input and returns the string with entities replaced with the corresponding labels 
    from Spacy package.
    '''
    doc = nlp(string)
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            string = string.replace(ent.text, 'famousperson')
    return string.lower()

def reduce_lengthening(text):
    '''Function that eliminates more than two repeat letters from a word.
    '''
    pattern = re.compile(r"(.)\1{2,}")
    return pattern.sub(r"\1\1", text)
result = string.punctuation 
def text_prepro(text):
    '''Function that performs text preprocessing like spelling correction, 'cleaning' a string from stop words, digits, 
    accents, one letter word; entities replacement, stemming, etc.
    '''
    stop_words = set(stopwords.words('english'))
    text = reduce_lengthening(text)
    review_text = BeautifulSoup(text).get_text()
    text = ent_replace(review_text)
    text = text.lower()
    text = text.replace("movie", "film").replace("ca n't", "cannot").replace("wo n't", "will not")
    text = text.replace("n't", "not").replace("!", "auxexcl")
    text = ''.join([l if l not in result else ' ' for l in text]).replace('  ', ' ')
    word_tokens = word_tokenize(text) 
    lemmatizer = WordNetLemmatizer()
    lem_n = [lemmatizer.lemmatize(word) for word in word_tokens]
    lem_a = [lemmatizer.lemmatize(word, pos='a') for word in lem_n]
    lem_v = [lemmatizer.lemmatize(word, pos='v') for word in lem_a]
    lemmed_words = [word for word in lem_v if word.isalpha() and (word not in stop_words or (word == 'not'or word == 'no')) and len(word)>1]
    lem_string = ' '.join(lemmed_words)
    return lem_string

In [ ]:
train['phrase_mod'] = train['Phrase'].apply(text_prepro)
test['phrase_modif'] = test['Phrase'].apply(text_prepro)

In [ ]:
cloud_comment_words = WordCloud(background_color="black").generate(
    ' '.join(list(train['phrase_mod'][train['Sentiment'] == 0])))
plt.imshow(cloud_comment_words, interpolation='bilinear') 
plt.axis("off")
plt.show()

In [ ]:
cloud_comment_words = WordCloud(background_color="black").generate(
    ' '.join(list(train['phrase_mod'][train['Sentiment'] == 1])))
plt.imshow(cloud_comment_words, interpolation='bilinear') 
plt.axis("off")
plt.show()

In [ ]:
cloud_comment_words = WordCloud(background_color="black").generate(
    ' '.join(list(train['phrase_mod'][train['Sentiment'] == 2])))
plt.imshow(cloud_comment_words, interpolation='bilinear') 
plt.axis("off")
plt.show()

In [ ]:
cloud_comment_words = WordCloud(background_color="black").generate(
    ' '.join(list(train['phrase_mod'][train['Sentiment'] == 3])))
plt.imshow(cloud_comment_words, interpolation='bilinear') 
plt.axis("off")
plt.show()

In [ ]:
cloud_comment_words = WordCloud(background_color="black").generate(
    ' '.join(list(train['phrase_mod'][train['Sentiment'] == 4])))
plt.imshow(cloud_comment_words, interpolation='bilinear') 
plt.axis("off")
plt.show()

In [ ]:
word_freq = Counter(chain.from_iterable(list(train['phrase_mod'].str.split())))
most_common = word_freq.most_common()

word_freq_t = Counter(chain.from_iterable(list(train['phrase_mod'].str.split())))
total = word_freq_t.most_common()

word_freq_0 = Counter(chain.from_iterable(list(train['phrase_mod'][train['Sentiment'] == 0].str.split())))
negative = word_freq_0.most_common()

word_freq_1 = Counter(chain.from_iterable(list(train['phrase_mod'][train['Sentiment'] == 1].str.split())))
mod_negative = word_freq_1.most_common()

word_freq_2 = Counter(chain.from_iterable(list(train['phrase_mod'][train['Sentiment'] == 2].str.split())))
neutral = word_freq_2.most_common()

word_freq_3 = Counter(chain.from_iterable(list(train['phrase_mod'][train['Sentiment'] == 3].str.split())))
mod_positive = word_freq_3.most_common()

word_freq_4 = Counter(chain.from_iterable(list(train['phrase_mod'][train['Sentiment'] == 4].str.split())))
positive = word_freq_4.most_common()

In [ ]:
negative[:10]

In [ ]:
mod_negative[:15]

In [ ]:
neutral[:15]

In [ ]:
mod_positive[:15]

In [ ]:
positive[:15]

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer_cv = CountVectorizer(ngram_range=(1, 3))
X_cv = vectorizer_cv.fit_transform(train['phrase_mod'])
y_cv = train['Sentiment']
X_train_cv, X_test_cv, y_train_cv, y_test_cv = train_test_split(X_cv, y_cv, test_size=0.1, 
                                                    stratify = y_cv, random_state=42)
print(X_train_cv.shape)
clf_cv = MultinomialNB()
clf_cv.fit(X_train_cv, y_train_cv)

prediction_cv = clf_cv.predict(X_test_cv)
print(accuracy_score(y_test_cv, prediction_cv))
cf_matrix_cv = confusion_matrix(y_test_cv, prediction_cv)
sns.heatmap(cf_matrix_cv/np.sum(cf_matrix_cv), annot=True, 
            fmt='.2%', cmap='Blues')

In [ ]:
class_labels = clf_cv.classes_

feature_names = vectorizer_cv.get_feature_names()

# Zip the feature names together with the coefficient array and sort by weights: feat_with_weights
feat_with_weights = sorted(zip(clf_cv.coef_[0], feature_names))

# Print the first class label and the top 20 feat_with_weights entries
print('Top 20 lowest values (less predictive features)')
print()
print(class_labels[0], feat_with_weights[:20])
print()
print('*******************************************************************************************************************************')
print()
# Print the second class label and the bottom 20 feat_with_weights entries
print('top 20 highest values (highest predictive features)')
print()
print(class_labels[1], feat_with_weights[-20:])

In [ ]:
matr = vectorizer_cv.transform(test['phrase_modif'])
test['Sentiment'] = clf_cv.predict(matr)

In [ ]:
test['Sentiment'].value_counts(normalize = True)

In [ ]:
train['Sentiment'].value_counts(normalize = True)

In [ ]:
submission = test[['PhraseId', 'Sentiment']]

In [ ]:
from IPython.display import FileLink

In [ ]:
submission.to_csv('mult_nom.csv', index=False)
FileLink(r'mult_nom.csv')